# Natural Language Processing Lab

In this lab we will further explore Scikit's and NLTK's capabilities to process text. We will use the 20 Newsgroup dataset, which is provided by Scikit-Learn.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:

categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

## 1. Data inspection

We have downloaded a few newsgroup categories and removed headers, footers and quotes.

Because this is an SKLearn dataset, it comes with pre-split train and test sets (note we were able to call 'train' and 'test' in subset).

Let's inspect them.

1. What data taype is `data_train`
> sklearn.datasets.base.Bunch
- Is it like a list? Or like a Dictionary? or what?
> Dict
- How many data points does it contain?
- Inspect the first data point, what does it look like?
> A blurb of text

In [4]:
type(data_train)

sklearn.utils.Bunch

In [5]:
data_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [6]:
len(data_train['data'])

2034

In [7]:
len(data_train['target'])

2034

In [34]:
data_train['target']

array([1, 3, 2, ..., 1, 0, 1])

In [8]:
data_train['data'][0]

"Hi,\n\nI've noticed that if you only save a model (with all your mapping planes\npositioned carefully) to a .3DS file that when you reload it after restarting\n3DS, they are given a default position and orientation.  But if you save\nto a .PRJ file their positions/orientation are preserved.  Does anyone\nknow why this information is not stored in the .3DS file?  Nothing is\nexplicitly said in the manual about saving texture rules in the .PRJ file. \nI'd like to be able to read the texture rule information, does anyone have \nthe format for the .PRJ file?\n\nIs the .CEL file format available from somewhere?\n\nRych"

## 2. Bag of Words model

Let's train a model using a simple count vectorizer

1. Initialize a standard CountVectorizer and fit the training data
- how big is the feature dictionary?
- repeat eliminating english stop words
- is the dictionary smaller?
- transform the training data using the trained vectorizer
- evaluate the performance of a Lotistic Regression on the features extracted by the CountVectorizer
    - you will have to transform the test_set too. Be carefule to use the trained vectorizer, without re-fitting it

BONUS:
- try a couple modifications:
    - restrict the max_features
    - change max_df and min_df

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
vect = CountVectorizer()
data_train_dtm = vect.fit_transform(data_train.data)

In [24]:
data_train_dtm

<2034x26879 sparse matrix of type '<class 'numpy.int64'>'
	with 196700 stored elements in Compressed Sparse Row format>

In [27]:
vect = CountVectorizer(stop_words = 'english')
data_train_dtm = vect.fit_transform(data_train.data)
data_train_dtm

<2034x26576 sparse matrix of type '<class 'numpy.int64'>'
	with 133634 stored elements in Compressed Sparse Row format>

In [29]:
vect.get_feature_names()[-50:]

['zien',
 'zijn',
 'zillion',
 'zillions',
 'zion',
 'zionist',
 'zip',
 'zipped',
 'zipping',
 'zippy',
 'zlumber',
 'zo',
 'zogeheten',
 'zombie',
 'zond',
 'zone',
 'zonker',
 'zoo',
 'zoology',
 'zoom',
 'zooming',
 'zooms',
 'zopfi',
 'zorastrian',
 'zorg',
 'zorn',
 'zoro',
 'zoroaster',
 'zoroastrian',
 'zoroastrianism',
 'zoroastrians',
 'zsoft',
 'zubin',
 'zuck',
 'zues',
 'zug',
 'zullen',
 'zulu',
 'zur',
 'zurbrin',
 'zurich',
 'zurvanism',
 'zus',
 'zvi',
 'zwaartepunten',
 'zwak',
 'zwakke',
 'zware',
 'zwarte',
 'zyxel']

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
lr = LogisticRegression()
lr.fit(data_train_dtm, data_train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [44]:
X_test = data_test.data
y_test = data_test.target

In [49]:
y_preds = lr.predict(vect.transform(X_test))

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
accuracy_score(y_test, y_preds)

0.74501108647450109

In [82]:
cv2 = CountVectorizer(max_features = 25000)
data_train2 = cv2.fit_transform(data_train.data)

In [83]:
lr2 = LogisticRegression()
lr2.fit(data_train2, data_train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [84]:
y_preds2 = lr2.predict(cv2.transform(X_test))
accuracy_score(y_test, y_preds2)

0.72135994087213595

## 3. Hashing and TF-IDF

Let's see if Hashing or TF-IDF improves the accuracy.

1. Initialize a HashingVectorizer and repeat the test with no restriction on the number of features
- does the score improve with respect to the count vectorizer?
- print out the number of features for this model
- Initialize a TF-IDF Vectorizer and repeat the analysis above
- print out the number of features for this model

BONUS
- Change the parameters of either (or both!) models to improve your score

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer

In [89]:
tfidfvect = TfidfVectorizer(max_df=0.8,min_df=0.2)
hashvect = HashingVectorizer()

In [90]:
datatrain3 = hashvect.fit_transform(data_train.data)
lr3 = LogisticRegression()
lr3.fit(datatrain3, data_train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [93]:
y_preds3 = lr3.predict(hashvect.transform(X_test))
accuracy_score(y_test, y_preds3)

0.67405764966740578

In [94]:
datatrain4 = tfidfvect.fit_transform(data_train.data)
lr4 = LogisticRegression()
lr4.fit(datatrain4, data_train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [95]:
y_preds4 = lr4.predict(tfidfvect.transform(X_test))
accuracy_score(y_test, y_preds4)

0.4493717664449372